In [1]:
from google.colab import files
uploaded = files.upload()

Saving spam.csv to spam.csv


In [2]:
import pandas as pd
import numpy as np

In [4]:
import gensim

In [67]:
df = pd.read_csv('/content/spam.csv', encoding='ISO-8859-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [69]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)

In [70]:
df.rename({'v1': 'category', 'v2': 'mail'}, axis=1, inplace=True)

In [71]:
df.head()

,category,mail
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [72]:
df.mail[0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [73]:
df.category.value_counts()

ham     4825
spam     747
Name: category, dtype: int64

In [15]:
mail_processed = df['mail'].apply(gensim.utils.simple_preprocess)

In [16]:
mail_processed[0]

['go',
 'until',
 'jurong',
 'point',
 'crazy',
 'available',
 'only',
 'in',
 'bugis',
 'great',
 'world',
 'la',
 'buffet',
 'cine',
 'there',
 'got',
 'amore',
 'wat']

In [23]:
cbow_model = gensim.models.Word2Vec(size=300, window=9, min_count=2)
sgram_model = gensim.models.Word2Vec(size=300, window=9, 
                                       min_count=2, sg=1)

In [24]:
cbow_model.build_vocab(mail_processed)

In [25]:
sgram_model.build_vocab(mail_processed)

In [26]:
cbow_model.corpus_count

5572

In [27]:
sgram_model.corpus_count

5572

In [28]:
cbow_model.epochs

5

In [29]:
sgram_model.epochs

5

In [30]:
cbow_model.train(mail_processed, total_examples=cbow_model.corpus_count, 
                 epochs=cbow_model.epochs)

(300692, 391395)

In [31]:
sgram_model.train(mail_processed, total_examples=sgram_model.corpus_count, 
                  epochs=sgram_model.epochs)

(300774, 391395)

In [33]:
print('cbow model vocabulary size:', len(cbow_model.wv.vocab))

print('skgram model vocabulary size:', len(sgram_model.wv.vocab))

cbow model vocabulary size: 3892
skgram model vocabulary size: 3892


In [34]:
cbow_model.wv.most_similar('great')

[('and', 0.9999648332595825),
 ('hope', 0.9999615550041199),
 ('miss', 0.9999595880508423),
 ('well', 0.9999584555625916),
 ('dear', 0.9999580383300781),
 ('see', 0.9999542236328125),
 ('as', 0.9999539256095886),
 ('of', 0.9999518394470215),
 ('hey', 0.9999500513076782),
 ('morning', 0.9999500513076782)]

In [35]:
sgram_model.wv.most_similar('great')

[('hope', 0.9581222534179688),
 ('princess', 0.9579008221626282),
 ('this', 0.9533922672271729),
 ('missing', 0.951522946357727),
 ('year', 0.950915515422821),
 ('love', 0.9486219882965088),
 ('thanks', 0.9473127126693726),
 ('semester', 0.9446690678596497),
 ('thank', 0.9437922239303589),
 ('had', 0.9370018243789673)]

In [46]:
for token in mail_processed[1]:
    if token in sgram_model.wv.vocab:
        print(token)

ok
lar
joking
wif
oni


In [42]:
(np.mean(sgram_model.wv.word_vec('go'), axis=0)).tolist()

0.003865443868562579

In [53]:
file_name = '/content/drive/MyDrive/Ml_course/NLP_sessions/GoogleNews-vectors-negative300.bin'

In [54]:
google_model = gensim.models.KeyedVectors.load_word2vec_format(file_name, binary=True)

In [55]:
def get_embedding_w2v(doc_tokens, pre_trained=0, cbow=0):
    '''doc_tokens: list of words,
    pre_trained: int 0 or 1; if 1 use google model
    cbow: int 0 or 1: if 1 use cbow; default is skipgram model'''
    embeddings = []
    if cbow:
        pre_trained = 0
    if pre_trained:
        model = google_model
    elif cbow:
        model = cbow_model
    else:
        model = sgram_model
    # iterate over tokens to extract their vectors    
    for tok in doc_tokens:
        if tok in model.wv.vocab:
            embeddings.append(model.wv.word_vec(tok))
    # mean the vectors of individual words to get the vector of the statement
    return np.mean(embeddings, axis=0)

In [63]:
google_vectors = mail_processed.apply(lambda x: get_embedding_w2v(x))
google_vectors = pd.DataFrame(google_vectors.tolist())
print('X shape:', google_vectors.shape)

TypeError: ignored

In [62]:
google_vectors

0       [-0.023970269, -0.08539971, 0.026500802, 0.046...
1       [-0.07740392, -0.0884452, 0.047572274, 0.05087...
2       [0.07374151, -0.08042842, 0.09463927, -0.06241...
3       [-0.09185543, -0.11664696, 0.05684878, 0.09473...
4       [-0.015822953, -0.110719144, 0.024480356, 0.08...
                              ...                        
5567    [0.07114995, -0.064738415, 0.0080614565, 0.039...
5568    [-0.039234262, -0.08500642, 0.055060048, 0.030...
5569    [-0.0022907848, -0.11255865, 0.0028063362, 0.0...
5570    [0.003552802, -0.1052001, 0.033180673, 0.04258...
5571    [0.002378621, -0.09587888, 0.009989627, 0.0359...
Name: mail, Length: 5572, dtype: object